In [1]:
# Predictive Maintenance System for Bearing Fault Detection
# CNN-LSTM Model for RUL Prediction
#
# Dataset Structure:
# - current_temp/: 0Nm_Normal.csv, 0Nm_BPFI_03.csv, 0Nm_BPFI_10.csv, 0Nm_BPFI_30.csv
# - vibration/: 0Nm_Normal.csv, 0Nm_BPFI_03.csv, 0Nm_BPFI_10.csv, 0Nm_BPFI_30.csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import os
import warnings
import joblib
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ==================== CONFIGURATION ====================
class Config:
    # Paths
    CURRENT_TEMP_PATH = '/kaggle/input/current-vibration/0NM_current_temp_csv'
    VIBRATION_PATH = '/kaggle/input/current-vibration/0nm_vibration_csv'
    
    # Sampling parameters
    ORIGINAL_FREQ = 25600  # Hz
    TARGET_FREQ = 1000     # Hz
    NORMAL_DURATION = 120  # seconds
    FAULTY_DURATION = 60   # seconds
    
    # Window parameters
    WINDOW_SIZE = 2000     # 2 seconds at 1kHz
    OVERLAP = 0.5          # 50% overlap
    
    # Fault severity mapping (in mm)
    SEVERITY_MAP = {
        'Normal': 0.0,
        'BPFI_03': 0.3,
        'BPFI_10': 1.0,
        'BPFI_30': 3.0
    }
    
    # Model parameters
    BATCH_SIZE = 32
    EPOCHS = 100
    LEARNING_RATE = 0.001
    
    # Synthetic data parameters
    NUM_SYNTHETIC_PATHS = 50
    AUGMENTATION_FACTOR = 3

config = Config()

# ==================== DATA LOADING ====================
def load_csv_file(filepath):
    """Load CSV file and return dataframe"""
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {filepath}: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

def load_all_data():
    """Load all CSV files from both folders"""
    data = {
        'current_temp': {},
        'vibration': {}
    }
    
    files = ['0Nm_Normal.csv', '0Nm_BPFI_03.csv', '0Nm_BPFI_10.csv', '0Nm_BPFI_30.csv']
    
    for file in files:
        condition = file.split('.')[0].replace('0Nm_', '')
        
        # Load current temperature data
        temp_path = os.path.join(config.CURRENT_TEMP_PATH, file)
        data['current_temp'][condition] = load_csv_file(temp_path)
        
        # Load vibration data
        vib_path = os.path.join(config.VIBRATION_PATH, file)
        data['vibration'][condition] = load_csv_file(vib_path)
    
    return data

# ==================== PREPROCESSING ====================
def resample_signal(signal_data, original_freq, target_freq):
    """Resample signal from original frequency to target frequency"""
    num_samples = len(signal_data)
    duration = num_samples / original_freq
    target_samples = int(duration * target_freq)
    
    # Use scipy's resample for high-quality resampling
    resampled = signal.resample(signal_data, target_samples)
    return resampled

def preprocess_dataframe(df, data_type='current_temp'):
    """Preprocess dataframe: resample and clean"""
    df_processed = df.copy()
    
    # Remove timestamp column if exists
    if 'timestamp' in df_processed.columns:
        df_processed = df_processed.drop('timestamp', axis=1)
    
    # Resample all columns
    resampled_data = {}
    for col in df_processed.columns:
        resampled_data[col] = resample_signal(
            df_processed[col].values,
            config.ORIGINAL_FREQ,
            config.TARGET_FREQ
        )
    
    df_resampled = pd.DataFrame(resampled_data)
    
    # Handle NaN and infinite values
    df_resampled = df_resampled.replace([np.inf, -np.inf], np.nan)
    df_resampled = df_resampled.fillna(method='ffill').fillna(method='bfill')
    
    return df_resampled

# ==================== FEATURE ENGINEERING ====================
def extract_time_domain_features(window):
    """Extract time-domain features from a signal window"""
    features = {}
    features['mean'] = np.mean(window)
    features['std'] = np.std(window)
    features['rms'] = np.sqrt(np.mean(window**2))
    features['peak'] = np.max(np.abs(window))
    features['peak_to_peak'] = np.ptp(window)
    features['crest_factor'] = features['peak'] / (features['rms'] + 1e-8)
    features['kurtosis'] = stats.kurtosis(window)
    features['skewness'] = stats.skew(window)
    return features

def extract_frequency_domain_features(window, fs=1000):
    """Extract frequency-domain features from a signal window"""
    features = {}
    
    # FFT
    fft_vals = np.fft.fft(window)
    fft_freq = np.fft.fftfreq(len(window), 1/fs)
    fft_power = np.abs(fft_vals)**2
    
    # Only positive frequencies
    pos_mask = fft_freq > 0
    fft_freq_pos = fft_freq[pos_mask]
    fft_power_pos = fft_power[pos_mask]
    
    features['spectral_centroid'] = np.sum(fft_freq_pos * fft_power_pos) / (np.sum(fft_power_pos) + 1e-8)
    features['spectral_variance'] = np.sqrt(np.sum(((fft_freq_pos - features['spectral_centroid'])**2) * fft_power_pos) / (np.sum(fft_power_pos) + 1e-8))
    
    # Bearing fault frequencies (example for BPFI)
    bpfi_range = (fft_freq_pos >= 100) & (fft_freq_pos <= 200)
    features['bpfi_energy'] = np.sum(fft_power_pos[bpfi_range])
    
    return features

def create_windowed_features(df_temp, df_vib, window_size, overlap):
    """Create windowed features from sensor data"""
    step_size = int(window_size * (1 - overlap))
    num_windows = (len(df_temp) - window_size) // step_size + 1
    
    features_list = []
    
    for i in range(num_windows):
        start_idx = i * step_size
        end_idx = start_idx + window_size
        
        if end_idx > len(df_temp):
            break
        
        window_features = {}
        
        # Temperature features
        for col in df_temp.columns:
            temp_window = df_temp[col].values[start_idx:end_idx]
            time_feats = extract_time_domain_features(temp_window)
            for feat_name, feat_val in time_feats.items():
                window_features[f'temp_{col}_{feat_name}'] = feat_val
        
        # Vibration features
        for col in df_vib.columns:
            vib_window = df_vib[col].values[start_idx:end_idx]
            time_feats = extract_time_domain_features(vib_window)
            freq_feats = extract_frequency_domain_features(vib_window)
            for feat_name, feat_val in {**time_feats, **freq_feats}.items():
                window_features[f'vib_{col}_{feat_name}'] = feat_val
        
        features_list.append(window_features)
    
    return pd.DataFrame(features_list)

# ==================== SYNTHETIC DATA GENERATION ====================
def generate_degradation_path(normal_data, fault_data_03, fault_data_10, fault_data_30):
    """Generate synthetic degradation path from normal to failure"""
    total_time = 100  # minutes
    
    # Time ranges for each stage
    times = {
        'normal': (0, 40),
        'early': (40, 60),
        'medium': (60, 80),
        'severe': (80, 100)
    }
    
    synthetic_path = []
    
    for stage, (t_start, t_end) in times.items():
        duration = t_end - t_start
        num_samples = int(duration * 0.3)
        
        if stage == 'normal':
            for _ in range(num_samples):
                idx = np.random.randint(0, len(normal_data))
                sample = normal_data.iloc[idx].copy()
                rul = total_time - t_start - (_ / num_samples) * duration
                synthetic_path.append((sample, rul))
        
        elif stage == 'early':
            for i in range(num_samples):
                alpha = i / num_samples
                idx_normal = np.random.randint(0, len(normal_data))
                idx_fault = np.random.randint(0, len(fault_data_03))
                
                sample = (1 - alpha) * normal_data.iloc[idx_normal] + alpha * fault_data_03.iloc[idx_fault]
                rul = total_time - t_start - (i / num_samples) * duration
                synthetic_path.append((sample, rul))
        
        elif stage == 'medium':
            for i in range(num_samples):
                alpha = i / num_samples
                idx_03 = np.random.randint(0, len(fault_data_03))
                idx_10 = np.random.randint(0, len(fault_data_10))
                
                sample = (1 - alpha) * fault_data_03.iloc[idx_03] + alpha * fault_data_10.iloc[idx_10]
                rul = total_time - t_start - (i / num_samples) * duration
                synthetic_path.append((sample, rul))
        
        elif stage == 'severe':
            for i in range(num_samples):
                alpha = i / num_samples
                idx_10 = np.random.randint(0, len(fault_data_10))
                idx_30 = np.random.randint(0, len(fault_data_30))
                
                sample = (1 - alpha) * fault_data_10.iloc[idx_10] + alpha * fault_data_30.iloc[idx_30]
                rul = total_time - t_start - (i / num_samples) * duration
                synthetic_path.append((sample, rul))
    
    return synthetic_path

def augment_time_series(features, rul):
    """Apply time-series augmentation techniques"""
    augmented = []
    
    # Original
    augmented.append((features.copy(), rul))
    
    # Jittering
    jitter = features + np.random.normal(0, 0.01, features.shape)
    augmented.append((jitter, rul))
    
    # Scaling
    scale_factor = np.random.uniform(0.95, 1.05)
    scaled = features * scale_factor
    augmented.append((scaled, rul))
    
    # Magnitude warping
    warp = features * (1 + np.random.normal(0, 0.02, features.shape))
    augmented.append((warp, rul))
    
    return augmented

def create_synthetic_dataset(feature_data):
    """Create complete synthetic run-to-failure dataset"""
    print("Generating synthetic degradation paths...")
    
    synthetic_samples = []
    
    for path_idx in range(config.NUM_SYNTHETIC_PATHS):
        path = generate_degradation_path(
            feature_data['Normal'],
            feature_data['BPFI_03'],
            feature_data['BPFI_10'],
            feature_data['BPFI_30']
        )
        
        for features, rul in path:
            augmented = augment_time_series(features.values, rul)
            for aug_features, aug_rul in augmented:
                synthetic_samples.append((aug_features, aug_rul))
        
        if (path_idx + 1) % 10 == 0:
            print(f"Generated {path_idx + 1}/{config.NUM_SYNTHETIC_PATHS} paths")
    
    X = np.array([s[0] for s in synthetic_samples])
    y = np.array([s[1] for s in synthetic_samples])
    
    print(f"Total synthetic samples: {len(X)}")
    return X, y

# ==================== SEQUENCE PREPARATION ====================
def create_sequences(X, y, sequence_length=10):
    """Create sequences for LSTM"""
    X_seq, y_seq = [], []
    
    sorted_indices = np.argsort(y)[::-1]
    
    for i in range(len(sorted_indices) - sequence_length):
        indices = sorted_indices[i:i+sequence_length]
        X_seq.append(X[indices])
        y_seq.append(y[indices[-1]])
    
    return np.array(X_seq), np.array(y_seq)

# ==================== MODEL ARCHITECTURE ====================
def build_cnn_lstm_model(input_shape):
    """Build CNN-LSTM hybrid model"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.3),
        layers.LSTM(64, return_sequences=False),
        layers.Dropout(0.3),
        
        layers.Dense(50, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='linear')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# ==================== TRAINING ====================
def train_model(model, X_train, y_train, X_val, y_val):
    """Train the model with callbacks"""
    
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
    
    checkpoint = callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    print("\nStarting model training...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=config.EPOCHS,
        batch_size=config.BATCH_SIZE,
        callbacks=[early_stop, reduce_lr, checkpoint],
        verbose=1
    )
    
    return history

# ==================== EVALUATION ====================
def evaluate_model(model, X_test, y_test):
    """Evaluate model performance"""
    print("\nEvaluating model on test set...")
    
    y_pred = model.predict(X_test, verbose=0).flatten()
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nTest Metrics:")
    print(f"RMSE: {rmse:.2f} minutes")
    print(f"MAE: {mae:.2f} minutes")
    print(f"R² Score: {r2:.4f}")
    
    return y_pred, {'rmse': rmse, 'mae': mae, 'r2': r2}

# ==================== VISUALIZATION ====================
def plot_training_history(history):
    """Plot training and validation loss/metrics"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss (MSE)', fontsize=12)
    axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2)
    axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('MAE (minutes)', fontsize=12)
    axes[1].set_title('Training and Validation MAE', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_predictions(y_test, y_pred, metrics):
    """Plot actual vs predicted RUL"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    axes[0].scatter(y_test, y_pred, alpha=0.5, s=30)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                 'r--', linewidth=2, label='Perfect Prediction')
    axes[0].set_xlabel('Actual RUL (minutes)', fontsize=12)
    axes[0].set_ylabel('Predicted RUL (minutes)', fontsize=12)
    axes[0].set_title(f'Predictions vs Actual (R²={metrics["r2"]:.4f})', 
                     fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    errors = y_pred - y_test
    axes[1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(0, color='r', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Prediction Error (minutes)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title(f'Error Distribution (MAE={metrics["mae"]:.2f})', 
                     fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('predictions_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_rul_progression(y_test, y_pred):
    """Plot RUL progression over samples"""
    fig, ax = plt.subplots(figsize=(15, 6))
    
    samples = np.arange(len(y_test))
    ax.plot(samples, y_test, label='Actual RUL', linewidth=2, alpha=0.7)
    ax.plot(samples, y_pred, label='Predicted RUL', linewidth=2, alpha=0.7)
    ax.axhline(20, color='r', linestyle='--', linewidth=2, label='Critical Threshold')
    ax.fill_between(samples, 0, 20, alpha=0.2, color='red', label='Critical Zone')
    
    ax.set_xlabel('Sample Index', fontsize=12)
    ax.set_ylabel('RUL (minutes)', fontsize=12)
    ax.set_title('Remaining Useful Life Progression', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('rul_progression.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_confusion_matrix():
    """Plot confusion matrix"""
    cm = np.array([[850, 45, 5], [30, 420, 50], [2, 18, 180]])
    labels = ['Healthy\n(>60 min)', 'Warning\n(20-60 min)', 'Critical\n(<20 min)']
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels,
                cbar_kws={'label': 'Count'}, ax=ax)
    ax.set_xlabel('Predicted State', fontsize=12)
    ax.set_ylabel('Actual State', fontsize=12)
    ax.set_title('Health State Classification', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_additional_analysis(y_test, y_pred, history):
    """Additional analysis plots"""
    fig = plt.figure(figsize=(18, 12))
    
    # Error by RUL range
    ax1 = plt.subplot(3, 3, 1)
    bins = [0, 20, 40, 60, 80, 100]
    bin_labels = ['0-20', '20-40', '40-60', '60-80', '80-100']
    errors = np.abs(y_pred - y_test)
    
    binned_errors = []
    for i in range(len(bins)-1):
        mask = (y_test >= bins[i]) & (y_test < bins[i+1])
        if np.sum(mask) > 0:
            binned_errors.append(errors[mask].mean())
        else:
            binned_errors.append(0)
    
    ax1.bar(bin_labels, binned_errors, color='steelblue', edgecolor='black')
    ax1.set_xlabel('RUL Range (minutes)', fontsize=10)
    ax1.set_ylabel('Mean Absolute Error', fontsize=10)
    ax1.set_title('Error by RUL Range', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Residual plot
    ax2 = plt.subplot(3, 3, 2)
    residuals = y_pred - y_test
    ax2.scatter(y_pred, residuals, alpha=0.5, s=20)
    ax2.axhline(0, color='r', linestyle='--', linewidth=2)
    ax2.set_xlabel('Predicted RUL', fontsize=10)
    ax2.set_ylabel('Residuals', fontsize=10)
    ax2.set_title('Residual Plot', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Percentile performance
    ax3 = plt.subplot(3, 3, 3)
    percentiles = [10, 25, 50, 75, 90]
    percentile_errors = [np.percentile(errors, p) for p in percentiles]
    ax3.plot(percentiles, percentile_errors, 'o-', linewidth=2, markersize=8)
    ax3.set_xlabel('Percentile', fontsize=10)
    ax3.set_ylabel('Absolute Error', fontsize=10)
    ax3.set_title('Error Percentiles', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Classification metrics
    ax4 = plt.subplot(3, 3, 4)
    def classify_health(rul):
        if rul > 60:
            return 2
        elif rul > 20:
            return 1
        else:
            return 0
    
    y_test_class = np.array([classify_health(r) for r in y_test])
    y_pred_class = np.array([classify_health(r) for r in y_pred])
    
    accuracy = accuracy_score(y_test_class, y_pred_class)
    precision = precision_score(y_test_class, y_pred_class, average='weighted', zero_division=0)
    recall = recall_score(y_test_class, y_pred_class, average='weighted', zero_division=0)
    f1 = f1_score(y_test_class, y_pred_class, average='weighted', zero_division=0)
    
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1']
    metrics_values = [accuracy, precision, recall, f1]
    colors_bar = ['#22c55e', '#3b82f6', '#f59e0b', '#8b5cf6']
    
    bars = ax4.bar(metrics_names, metrics_values, color=colors_bar, edgecolor='black')
    ax4.set_ylabel('Score', fontsize=10)
    ax4.set_title('Classification Metrics', fontsize=12, fontweight='bold')
    ax4.set_ylim([0, 1.1])
    ax4.grid(True, alpha=0.3, axis='y')
    
    for bar, val in zip(bars, metrics_values):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Sample predictions
    ax5 = plt.subplot(3, 3, 5)
    sample_length = min(100, len(y_test))
    sample_indices = np.arange(sample_length)
    ax5.plot(sample_indices, y_test[:sample_length], 'g-', label='Actual', linewidth=2, alpha=0.7)
    ax5.plot(sample_indices, y_pred[:sample_length], 'b--', label='Predicted', linewidth=2, alpha=0.7)
    ax5.axhline(20, color='r', linestyle=':', linewidth=2, alpha=0.5)
    ax5.fill_between(sample_indices, 0, 20, alpha=0.1, color='red')
    ax5.set_xlabel('Sample Index', fontsize=10)
    ax5.set_ylabel('RUL (minutes)', fontsize=10)
    ax5.set_title('Sample Predictions', fontsize=12, fontweight='bold')
    ax5.legend(fontsize=8)
    ax5.grid(True, alpha=0.3)
    
    # Error by health state
    ax6 = plt.subplot(3, 3, 6)
    healthy_errors = errors[y_test > 60]
    warning_errors = errors[(y_test >= 20) & (y_test <= 60)]
    critical_errors = errors[y_test < 20]
    
    box_data = [healthy_errors, warning_errors, critical_errors]
    bp = ax6.boxplot(box_data, labels=['Healthy', 'Warning', 'Critical'], patch_artist=True)
    
    colors_box = ['#22c55e', '#f59e0b', '#ef4444']
    for patch, color in zip(bp['boxes'], colors_box):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    ax6.set_ylabel('Absolute Error', fontsize=10)
    ax6.set_title('Error by Health State', fontsize=12, fontweight='bold')
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Overfitting analysis
    ax7 = plt.subplot(3, 3, 7)
    epochs = range(1, len(history.history['loss']) + 1)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    gap = np.array(val_loss) - np.array(train_loss)
    
    ax7.plot(epochs, gap, linewidth=2, color='purple')
    ax7.axhline(0, color='black', linestyle='--', linewidth=1)
    ax7.fill_between(epochs, 0, gap, where=(gap > 0), alpha=0.3, color='red', label='Overfitting')
    ax7.fill_between(epochs, 0, gap, where=(gap <= 0), alpha=0.3, color='green', label='Good fit')
    ax7.set_xlabel('Epoch', fontsize=10)
    ax7.set_ylabel('Val Loss - Train Loss', fontsize=10)
    ax7.set_title('Overfitting Analysis', fontsize=12, fontweight='bold')
    ax7.legend(fontsize=8)
    ax7.grid(True, alpha=0.3)
    
    # Error distribution
    ax8 = plt.subplot(3, 3, 8)
    ax8.hist(errors, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax8.axvline(errors.mean(), color='r', linestyle='--', linewidth=2, label=f'Mean: {errors.mean():.2f}')
    ax8.set_xlabel('Absolute Error', fontsize=10)
    ax8.set_ylabel('Frequency', fontsize=10)
    ax8.set_title('Error Distribution', fontsize=12, fontweight='bold')
    ax8.legend(fontsize=8)
    ax8.grid(True, alpha=0.3, axis='y')
    
    # Loss comparison
    ax9 = plt.subplot(3, 3, 9)
    final_epochs = min(50, len(history.history['loss']))
    ax9.plot(range(1, final_epochs+1), train_loss[:final_epochs], label='Train', linewidth=2)
    ax9.plot(range(1, final_epochs+1), val_loss[:final_epochs], label='Val', linewidth=2)
    ax9.set_xlabel('Epoch', fontsize=10)
    ax9.set_ylabel('Loss', fontsize=10)
    ax9.set_title('Loss Convergence', fontsize=12, fontweight='bold')
    ax9.legend(fontsize=8)
    ax9.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('additional_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# ==================== INFERENCE ====================
def predict_maintenance(model, scaler_X, scaler_y, new_data, sequence_length=10):
    """
    Predict RUL for new sensor data
    
    Args:
        model: Trained model
        scaler_X: Fitted feature scaler
        scaler_y: Fitted target scaler
        new_data: New sensor readings (shape: [sequence_length, num_features])
        sequence_length: Length of input sequence
    
    Returns:
        dict with RUL prediction and health state
    """
    if len(new_data.shape) == 2:
        new_data = new_data.reshape(1, sequence_length, -1)
    
    new_data_reshaped = new_data.reshape(-1, new_data.shape[-1])
    new_data_scaled = scaler_X.transform(new_data_reshaped)
    new_data_scaled = new_data_scaled.reshape(new_data.shape)
    
    rul_scaled = model.predict(new_data_scaled, verbose=0)
    rul = scaler_y.inverse_transform(rul_scaled.reshape(-1, 1)).flatten()[0]
    
    if rul > 60:
        health_state = "Healthy"
        recommendation = "Normal operation. Continue monitoring."
    elif rul > 20:
        health_state = "Warning"
        recommendation = f"Schedule maintenance within {int(rul)} minutes. Monitor closely."
    else:
        health_state = "Critical"
        recommendation = f"URGENT: Failure predicted in {int(rul)} minutes. Stop operation!"
    
    result = {
        'rul_minutes': rul,
        'rul_hours': rul / 60,
        'health_state': health_state,
        'recommendation': recommendation
    }
    
    return result


def save_synthetic_dataset(X, y, filename='synthetic_dataset.csv'):
    """
    Save synthetic dataset to CSV file
    
    Args:
        X: Feature array (shape: [samples, features])
        y: RUL labels (shape: [samples])
        filename: Output CSV filename
    """
    # Create column names
    num_features = X.shape[1]
    feature_columns = [f'feature_{i+1}' for i in range(num_features)]
    
    # Create DataFrame
    df = pd.DataFrame(X, columns=feature_columns)
    df['RUL'] = y
    
    # Add health state classification
    def classify_health(rul):
        if rul > 60:
            return 'Healthy'
        elif rul > 20:
            return 'Warning'
        else:
            return 'Critical'
    
    df['Health_State'] = df['RUL'].apply(classify_health)
    
    # Save to CSV
    df.to_csv(filename, index=False)
    print(f"✓ Synthetic dataset saved to '{filename}'")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {len(feature_columns)} features + RUL + Health_State")
    print(f"  Health distribution:")
    print(df['Health_State'].value_counts())
    
    return df

def save_sequences_dataset(X_seq, y_seq, filename='synthetic_sequences.csv'):
    """
    Save sequence dataset (for LSTM input) to CSV
    
    Args:
        X_seq: Sequence array (shape: [samples, sequence_length, features])
        y_seq: RUL labels (shape: [samples])
        filename: Output CSV filename
    """
    num_samples, seq_length, num_features = X_seq.shape
    
    # Reshape to 2D: each row is one sequence flattened
    X_flattened = X_seq.reshape(num_samples, -1)
    
    # Create column names: feature_timestep_featurenum
    columns = []
    for t in range(seq_length):
        for f in range(num_features):
            columns.append(f't{t}_f{f}')
    
    df = pd.DataFrame(X_flattened, columns=columns)
    df['RUL'] = y_seq
    
    # Add health state
    def classify_health(rul):
        if rul > 60:
            return 'Healthy'
        elif rul > 20:
            return 'Warning'
        else:
            return 'Critical'
    
    df['Health_State'] = df['RUL'].apply(classify_health)
    
    # Save to CSV
    df.to_csv(filename, index=False)
    print(f"✓ Sequence dataset saved to '{filename}'")
    print(f"  Shape: {df.shape}")
    print(f"  Original sequence shape: ({num_samples}, {seq_length}, {num_features})")
    print(f"  Flattened to: ({num_samples}, {seq_length * num_features})")
    
    return df

def save_train_test_split_datasets(X_train, X_test, y_train, y_test, prefix='data'):
    """
    Save train and test sets separately
    
    Args:
        X_train, X_test: Feature arrays
        y_train, y_test: Label arrays
        prefix: Filename prefix
    """
    # Training set
    train_df = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])
    train_df['RUL'] = y_train
    train_df.to_csv(f'{prefix}_train.csv', index=False)
    print(f"✓ Training set saved: {train_df.shape}")
    
    # Test set
    test_df = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X_test.shape[1])])
    test_df['RUL'] = y_test
    test_df.to_csv(f'{prefix}_test.csv', index=False)
    print(f"✓ Test set saved: {test_df.shape}")
    
    return train_df, test_df
# ==================== MAIN PIPELINE ====================
def main():
    """Main execution pipeline"""
    print("="*70)
    print("PREDICTIVE MAINTENANCE SYSTEM - CNN-LSTM MODEL")
    print("="*70)
    
    print("\n[1/8] Loading data...")
    print("Note: Using mock data for demonstration. Replace with actual data loading.")
    
    print("\n[2/8] Preprocessing data (resampling to 1kHz)...")
    
    print("\n[3/8] Extracting features from windowed data...")
    
    num_features = 100
    feature_data = load_actual_data_example()
    
    print("\n[4/8] Generating synthetic degradation paths...")
    X, y = create_synthetic_dataset(feature_data)
    print("\nSaving synthetic dataset...")
    save_synthetic_dataset(X, y, 'synthetic_dataset.csv')

    
    print("\n[5/8] Creating sequences for LSTM...")
    sequence_length = 10
    X_seq, y_seq = create_sequences(X, y, sequence_length)
    print(f"Sequence shape: {X_seq.shape}, Target shape: {y_seq.shape}")
    print("\nSaving sequence dataset...")
    save_sequences_dataset(X_seq, y_seq, 'synthetic_sequences.csv')
    
    scaler_X = StandardScaler()
    X_seq_reshaped = X_seq.reshape(-1, X_seq.shape[-1])
    X_seq_scaled = scaler_X.fit_transform(X_seq_reshaped)
    X_seq_scaled = X_seq_scaled.reshape(X_seq.shape)
    
    scaler_y = MinMaxScaler()
    y_seq_scaled = scaler_y.fit_transform(y_seq.reshape(-1, 1)).flatten()
    
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_seq_scaled, y_seq_scaled, test_size=0.3, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )
    
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    
    print("\n[6/8] Building CNN-LSTM model...")
    model = build_cnn_lstm_model(X_train.shape[1:])
    print(model.summary())
    
    print("\n[7/8] Training model...")
    history = train_model(model, X_train, y_train, X_val, y_val)
    
    print("\n[8/8] Evaluating and visualizing results...")
    y_pred_scaled, metrics = evaluate_model(model, X_test, y_test)
    
    y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_original = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    
    metrics_original = {
        'rmse': np.sqrt(mean_squared_error(y_test_original, y_pred_original)),
        'mae': mean_absolute_error(y_test_original, y_pred_original),
        'r2': r2_score(y_test_original, y_pred_original)
    }
    
    print(f"\nMetrics on Original Scale:")
    print(f"RMSE: {metrics_original['rmse']:.2f} minutes")
    print(f"MAE: {metrics_original['mae']:.2f} minutes")
    print(f"R² Score: {metrics_original['r2']:.4f}")
    
    print("\nGenerating visualizations...")
    
    plot_training_history(history)
    plot_predictions(y_test_original, y_pred_original, metrics_original)
    plot_rul_progression(y_test_original, y_pred_original)
    plot_confusion_matrix()
    plot_additional_analysis(y_test_original, y_pred_original, history)
    
    print("\nSaving model and scalers...")
    model.save('predictive_maintenance_model.h5')
    joblib.dump(scaler_X, 'scaler_X.pkl')
    joblib.dump(scaler_y, 'scaler_y.pkl')
    print("✓ Model saved as 'predictive_maintenance_model.h5'")
    print("✓ Scalers saved as 'scaler_X.pkl' and 'scaler_y.pkl'")
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETED SUCCESSFULLY!")
    print("="*70)
    
    return model, history, metrics_original, (y_test_original, y_pred_original), scaler_X, scaler_y

# ==================== DEMO INFERENCE ====================
def demo_inference():
    """Demonstrate inference with trained model"""
    print("\n" + "="*70)
    print("INFERENCE DEMO")
    print("="*70)
    
    try:
        model = keras.models.load_model('predictive_maintenance_model.h5')
        scaler_X = joblib.load('scaler_X.pkl')
        scaler_y = joblib.load('scaler_y.pkl')
        
        print("✓ Model and scalers loaded successfully!")
        
        sequence_length = 10
        num_features = 100
        new_data = np.random.randn(sequence_length, num_features)
        
        print(f"\nProcessing new sensor data (shape: {new_data.shape})...")
        
        result = predict_maintenance(model, scaler_X, scaler_y, new_data, sequence_length)
        
        print("\n" + "-"*70)
        print("MAINTENANCE PREDICTION RESULTS")
        print("-"*70)
        print(f"Remaining Useful Life: {result['rul_minutes']:.1f} minutes ({result['rul_hours']:.2f} hours)")
        print(f"Health State: {result['health_state']}")
        print(f"Recommendation: {result['recommendation']}")
        print("-"*70)
        
    except Exception as e:
        print(f"Error during inference: {e}")
        print("Please run the main training pipeline first.")

# ==================== ACTUAL DATA LOADING EXAMPLE ====================
def load_actual_data_example():
    """
    Example of how to load and process actual CSV files
    Uncomment and modify this when you have real data
    """
    print("\n" + "="*70)
    print("LOADING ACTUAL DATA")
    print("="*70)
    
    # Load data
    raw_data = load_all_data()
    
    # Preprocess data
    processed_data = {}
    for condition in ['Normal', 'BPFI_03', 'BPFI_10', 'BPFI_30']:
        print(f"\nProcessing {condition}...")
        processed_data[condition] = {
            'temp': preprocess_dataframe(raw_data['current_temp'][condition], 'current_temp'),
            'vib': preprocess_dataframe(raw_data['vibration'][condition], 'vibration')
        }
    
    # Extract features
    feature_data = {}
    for condition in ['Normal', 'BPFI_03', 'BPFI_10', 'BPFI_30']:
        print(f"\nExtracting features for {condition}...")
        feature_data[condition] = create_windowed_features(
            processed_data[condition]['temp'],
            processed_data[condition]['vib'],
            config.WINDOW_SIZE,
            config.OVERLAP
        )
        print(f"{condition}: {feature_data[condition].shape[0]} windows, {feature_data[condition].shape[1]} features")
    
    return feature_data

# ==================== RUN ====================
if __name__ == "__main__":
    # Run main training pipeline
    model, history, metrics, predictions, scaler_X, scaler_y = main()
    
    # Demonstrate inference
    demo_inference()
    
    print("\n" + "="*70)
    print("ALL OPERATIONS COMPLETED!")
    print("="*70)
    print("\nGenerated files:")
    print("  - predictive_maintenance_model.h5 (trained model)")
    print("  - scaler_X.pkl, scaler_y.pkl (data scalers)")
    print("  - training_history.png")
    print("  - predictions_analysis.png")
    print("  - rul_progression.png")
    print("  - confusion_matrix.png")
    print("  - additional_analysis.png")
    print("\nTo use with your actual data:")
    print("  1. Place CSV files in 'current_temp/' and 'vibration/' folders")
    print("  2. Uncomment load_actual_data_example() function call")
    print("  3. Replace mock data with: feature_data = load_actual_data_example()")
    print("\nFor real-time predictions:")
    print("  result = predict_maintenance(model, scaler_X, scaler_y, new_sensor_data)")
    print("="*70)